This code chooses random frames from each track in a given video and creates a 5-second video of that tracked individual starting at that frame. It saves the video using a filename that encodes the observation, individual and starting frame number. These videos will then be classified as either stationary, walking or running.

Note that throughout this notebook, the frame numbers are in relation to the first frame of the observation (when the drone is in position above the animals), NOT in relation to the first frame of the raw drone video (which can begin before the drone takes off).

In [1]:
# import necessary packages
import os
import pandas as pd
import numpy as np
import glob
import random
import argparse
import cv2
import shutil
from tqdm.notebook import tqdm

mount = '/home/blair/server/herd_hover'
main_folder = os.path.join(mount, 'processing','kenya-tracking', 'processed-videos', 'raw-footage')

In [2]:
# set observation number and read in 2-sec subsetted tracks file
t = '027'
file = 'tracks-for-segmentation/ob' + t + '_sub2sec.csv'
tracks = pd.read_csv(file)

In [3]:
# pick random frames from each track. The number of random frames is the number of subsetted frames divided by 10 with a max of 30
rand_frames = {}
for i in tqdm(np.unique(tracks['trackID'])):
    allframes = tracks[tracks['trackID'] == i]['time'].values
    n_frames = int(round(len(allframes)/10,0))
    if n_frames > 30: # maximum of 30 videos for each track
        n_frames = 30
    picks = np.random.choice(allframes, n_frames, replace = False)
    rand_frames[i] = picks

  0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
# For each random frame, copy it and the next 150 frames into an individual folder on the server
for i in tqdm(rand_frames):
    # get observation and track numbers
    ob = 'observation' + i.split('b')[1].split('-')[0]
    #ob_num = i.split(
    track = i.split('0')[-1:]
    if len(track[0]) == 0:
        track = ['0']
    track_num = 'track-' + track[0]
    # set folder containing individual images 
    crop_folder = os.path.join(main_folder, ob, 'individual_images', track_num)
    # # get and sort list of all images in the crop folder and sub-folders
    # jpg_list = glob.glob(crop_folder+'/**/*.jpg', recursive = True)
    # jpg_list.sort()
   
    # take the first random frame and copy it along with the next 150 frames into a folder
    for p in tqdm(rand_frames[i]):
         # create a folder to hold frames for the video
        new_folder = os.path.join('groundtruth-clips', ob, track_num, str('startframe_' + str(p).zfill(5)))
        if not os.path.exists(new_folder):
            os.makedirs(new_folder)
        for t in np.arange(p, p+150):
            frame = 'frame_' + track[0].zfill(2) + '_' + str(t).zfill(5) + '.jpg'
            pattern = crop_folder + '/**/' + frame
            for fname in glob.glob(pattern, recursive=True):
                shutil.copy(fname, os.path.join(new_folder, frame))

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
## For each folder created in previous step, go through and combine frames into a video. Work in random order within a given track.

# First need to check that there are 150 frames in the folder (may not be the case for clips starting near the end of the ob).
# Delete any folders with fewer than 150 frames

# Need to check that clips don't overlap other clips for that same individual. As I score videos, make a list of starting/stopping 
# frames for each video; if a new clip will overlap these ranges, stop and move to next video.

In [6]:
frame_folders = glob.glob('./groundtruth-clips/*/*/*/', recursive = True)
video_folder = os.path.join(mount, 'vigilance', 'groundtruth-clips')
for i in tqdm(frame_folders):
    image_folder = i
    video_name = video_folder + '/' + ('_').join(i.split('/')[2:5]) + '.mp4'
    if not os.path.exists(os.path.join(video_name)):
        images = [img for img in os.listdir(image_folder) if img.endswith('.jpg')]
        # check if there are 150 frames (if random frame was too close to end of observation, the clip may
        # not be 5 seconds long and should be skipped.
        if len(images) == 150:
            images.sort()
            frame = cv2.imread(os.path.join(image_folder, images[0]))
            height, width, layers = frame.shape
            fps = 30
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video = cv2.VideoWriter(video_name, fourcc, fps, (width,height))
            for image in images:
                video.write(cv2.imread(os.path.join(image_folder, image)))
            cv2.destroyAllWindows()
            video.release()

  0%|          | 0/1207 [00:00<?, ?it/s]